# Introduction

To understand the data present in `search_history.json`, we
- Wrote `extract_schema` and `merge_schemas`: a mapping and reducer function that follows the mapReduce pattern to batch process 
  search_history.json entries in parallel to extract schema fields.
- We check for any falsy values; that is, empty lists, empty dictionaries, empty strings, and whitespace only results.
- We randomly sampled different entries to infer what the optional fields did; this was done as there was no documentation online for what
  each optional field did.

The search history data we are given in `search_history.json` is a Google Takeout export of search activity containing 55383 search entries
accumulated over 7 years from June 2017 to June 2024 with no falsy values. The lists below describe the schema and the meaning of both mandatory and optional fields in the schema.

**Mandatory Fields**

- `header` (str): The Google product category. Only `"Search"` was observed across all entries.
- `title` (str): Description of the activity. The following templates have been observed:

  | Template                   | Meaning                                                    |
  |----------------------------|------------------------------------------------------------|
  | "Searched for ..."         | User ran a Google search                                   |
  | "Visited ..."              | User visited a website from search results                 |
  | "Viewed ..."               | User clicked on a Google Maps entry from search results    |
  | "1 notification"           | User received a notification from Google Alerts            |
  | "Used Search"              | Unknown - appears sparsely (170 times)                     |
  | "Ran internet speed test"  | Unknown - appears only once                                |

  Here, the "..." is either a search string or a URL.
- `time` (str): ISO 8601 timestamp of when the activity occured. In the data, it ranged from `2017-06-08 16:42:55.223000+00:00` to
  `2024-06-23 22:21:50.431000+00:00` for a total duration of 2572 days (7.0 years).
- `products` (list): Google products involved. Only `["Search"]` was observed across all entries.
- `activityControls` (list): Which activity control settings captured this data. Only `["Web & App Activity"]` was observed across all entries.

**Optional Fields**

- `titleUrl` (str, 55,021 / 99.3%): The URL associated with the activity. In the data, it only appeared for the `title` templates "Searched
  for ..." and "Visited ...".
- `details` (list, 2,794 / 5.0%): Additional details about the activity. In the data, when the field was present, only `[{'name': 'From Google Ads'}]` was observed, indicating that the search result the user interacted with was recommended by Google Ads.
- `locationInfos` (list, 226 / 0.4%): Location data when search was made. This field only appears when you give Google permission to access 
    your location for the device you are performing the search for. The following location sources were observed:

  | source                       | count |
  |------------------------------|-------|
  | "From your places (Home)"    | 175   |
  | "From your places (Work)"    | 50    |
  | "Based on your past activity"| 1     |

  Here, "From your places" specifically refers to the "Labelled" location lists in Google Maps indicating your work and home locations. 
  In the data specifically, when the field was present, only the following values were observed:

  ```json
  // From your places (Home)
  {
    "name": "At this general area",
    "url": "https://www.google.com/maps/@?api=1&map_action=map&center=51.504495,-0.011733&zoom=12",
    "source": "From your places (Home)",
    "sourceUrl": "https://support.google.com/maps/answer/3184808"
  }

  // From your places (Work)
  {
    "name": "At this general area",
    "url": "https://www.google.com/maps/@?api=1&map_action=map&center=51.525493,-0.082217&zoom=12",
    "source": "From your places (Work)",
    "sourceUrl": "https://support.google.com/maps/answer/3184808"
  }

  // Based on your past activity
  {
    "name": "At this general area",
    "url": "https://www.google.com/maps/@?api=1&map_action=map&center=51.504467,-0.082155&zoom=12",
    "source": "Based on your past activity"
  }
  ```
- `subtitles` (list, 166 / 0.3%): Additional context for the search. The only time it appears is for the `title` "1 notification" and
  "Ran internet speed test". A breakdown of the subfields within `subtitles` are as follows:
  - `subtitles[].name` (str): Subtitle text (e.g., topic names like "Reuters")
  - `subtitles[].url` (str, 8): Optional URL within subtitle

In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor
from functools import reduce


def extract_schema(obj, prefix=""):
    """Map: extract all key paths and types from a dict."""
    paths = {}
    match obj:
        case dict(d):
            for k, v in d.items():
                path = f"{prefix}.{k}" if prefix else k
                paths[path] = {"types": {type(v).__name__}, "count": 1}
                nested = extract_schema(v, path)
                for p, info in nested.items():
                    paths[p] = info
        case list(items) if items:
            paths[f"{prefix}[]"] = {"types": {"list"}, "count": 1}
            nested = extract_schema(items[0], f"{prefix}[]")
            for p, info in nested.items():
                paths[p] = info
        case list():
            paths[f"{prefix}[]"] = {"types": {"list"}, "count": 1}
    return paths


def merge_schemas(a, b):
    """Reduce: merge two schemas, combining types and counts."""
    result = dict(a)
    for k, v in b.items():
        if k in result:
            result[k] = {
                "types": result[k]["types"] | v["types"],
                "count": result[k]["count"] + v["count"],
            }
        else:
            result[k] = v
    return result


def find_empty_values(obj, prefix=""):
    """Map: find all empty/null values using structural pattern matching."""
    issues = {}
    match obj:
        case None:
            issues[f"{prefix} (null)"] = 1
        case "":
            issues[f"{prefix} (empty string)"] = 1
        case str(s) if s.isspace():
            issues[f"{prefix} (whitespace only)"] = 1
        case list() if not obj:
            issues[f"{prefix} (empty list)"] = 1
        case dict() if not obj:
            issues[f"{prefix} (empty dict)"] = 1
        case dict(d):
            for k, v in d.items():
                path = f"{prefix}.{k}" if prefix else k
                nested = find_empty_values(v, path)
                for p, count in nested.items():
                    issues[p] = issues.get(p, 0) + count
        case list(items):
            for item in items:
                nested = find_empty_values(item, f"{prefix}[]")
                for p, count in nested.items():
                    issues[p] = issues.get(p, 0) + count
        case _:
            pass  # Non-empty primitive value
    return issues


def merge_issues(a, b):
    """Reduce: merge two issue dicts, summing counts."""
    result = dict(a)
    for k, v in b.items():
        result[k] = result.get(k, 0) + v
    return result


if __name__ == "__main__":
    with open("../src/search_history.json", "r") as f:
        data = json.load(f)

    total = len(data)
    print(f"Processing {total} entries...\n")

    with ThreadPoolExecutor() as executor:
        schemas = list(executor.map(extract_schema, data))
        empty_values = list(executor.map(find_empty_values, data))

    # Schema analysis
    full_schema = reduce(merge_schemas, schemas, {})

    mandatory = []
    optional = []

    for path in sorted(full_schema.keys()):
        info = full_schema[path]
        types = ", ".join(sorted(info["types"]))
        count = info["count"]

        if count == total:
            mandatory.append((path, types))
        else:
            optional.append((path, types, count))

    print("=== MANDATORY FIELDS (100%) ===")
    for path, types in mandatory:
        print(f"  {path}: {types}")

    print(f"\n=== OPTIONAL FIELDS ===")
    for path, types, count in optional:
        print(f"  {path}: {types} ({count}/{total})")

    # Empty values analysis
    combined = reduce(merge_issues, empty_values, {})

    if combined:
        print("\n=== EMPTY/NULL VALUES FOUND ===")
        for path, count in sorted(combined.items(), key=lambda x: -x[1]):
            print(f"  {path}: {count} occurrences")
    else:
        print("\nNo empty or null values found.")

In [ ]:
import json
import random

OPTIONAL_FIELDS = ["titleUrl", "details", "locationInfos", "subtitles"]
SAMPLE_SIZE = 5


def sample_by_field(data, field, n=SAMPLE_SIZE):
    """Get random sample of entries containing the specified field."""
    with_field = [e for e in data if field in e]
    without_field = [e for e in data if field not in e]
    return {
        "with": random.sample(with_field, min(n, len(with_field))),
        "without": random.sample(without_field, min(n, len(without_field))),
    }


def format_samples(samples, field):
    """Format samples for a field as a single string."""
    with_entries = "\n".join(
        f"""
Sample {i}:
  title: {entry.get('title', 'N/A')}
  {field}: {entry.get(field)}"""
        for i, entry in enumerate(samples["with"], 1)
    )

    without_entries = "\n".join(
        f"""
Sample {i}:
  title: {entry.get('title', 'N/A')}"""
        for i, entry in enumerate(samples["without"], 1)
    )

    return f"""
{'='*60}
FIELD: {field}
{'='*60}

--- Entries WITH {field} ---
{with_entries}

--- Entries WITHOUT {field} ---
{without_entries}
"""


if __name__ == "__main__":
    with open("../src/search_history.json", "r") as f:
        data = json.load(f)

    for field in OPTIONAL_FIELDS:
        samples = sample_by_field(data, field)
        print(format_samples(samples, field))

In [ ]:
import json
from collections import Counter
from datetime import datetime

CATEGORICAL_FIELDS = ["header", "products", "activityControls"]


def summarise_categorical(data, field):
    """Count occurrences of each unique value for a categorical field."""
    values = []
    for entry in data:
        val = entry.get(field)
        if isinstance(val, list):
            values.extend(val)
        else:
            values.append(val)
    return Counter(values)


def summarise_time(data):
    """Find min and max timestamps."""
    times = [datetime.fromisoformat(e["time"].replace("Z", "+00:00")) for e in data]
    return min(times), max(times)


if __name__ == "__main__":
    with open("../src/search_history.json", "r") as f:
        data = json.load(f)

    for field in CATEGORICAL_FIELDS:
        counts = summarise_categorical(data, field)
        print(f"""
{'='*60}
FIELD: {field} ({len(counts)} unique values)
{'='*60}
{chr(10).join(f'  {val}: {count}' for val, count in counts.most_common())}
""")

    min_time, max_time = summarise_time(data)
    duration = max_time - min_time
    print(f"""
{'='*60}
FIELD: time
{'='*60}
  Min: {min_time}
  Max: {max_time}
  Duration: {duration.days} days ({duration.days / 365:.1f} years)
""")

# Storing search history in a simplicial complex.

The 55,383 search entries spanning 7 years are semi-structured raw data. To process that into structured knowledge that captures user characteristics such as
- *what* the user was interested in and *how* those interests relate.
- a coherent narrative of the user's life and their tastes, values, etc.
we store it in a simplicial complex via a simplex tree. We have attached a paper justifying the use of the simplicial complex and the high level overview of the how retrieval and query pipelines are implemented for such a structure. This Python notebook primarily focuses on what a practical implementation of those pipelines could look like.

## The Extraction Pipeline

The extraction pipeline (`src/pipeline.py`) processes each search entry through four stages:
1. The `title` field encodes both activity type and content. We use pattern matching to parse and extract it:

  | Title Pattern | Activity Type | Content |
  |---------------|---------------|---------|
  | "Searched for jodhpur hotels" | `searched` | "jodhpur hotels" |
  | "Visited https://example.com" | `visited` | "https://example.com" |
  | "Viewed Umaid Bhawan Palace" | `viewed` | "Umaid Bhawan Palace" |
  | "1 notification" | `notification` | (extracted from subtitles) |

2. For each content string, we extract entities and relationships. We call 
  the DedalusLabs SDK with a structured prompt:

  ```
  Given this search activity: "jodhpur hotels"
  Extract:
  - Entities mentioned (proper nouns, places, concepts)
  - Relationships between entities (subject, predicate, object)
  ```

  The LLM returns structured output:
  - **Entities**: `["Jodhpur", "hotels"]`
  - **Relationships**: `[("hotels", "located_in", "Jodhpur")]`
  and each entity becomes a vertex with an embedding, while each relationship ecomes a directed edge. Here, we use the `openai/text-embedding-3-small` due to time and cost considerations.

3. We build simplexes for timestamp and location using witness complexes by 
  grouping entries by temporal and location proximity. Entries within a 30-minute window are assumed to be part of the same search session. For example, suppose in a 30 minute window, the user has the following search queries:
  - "jodhpur hotels", which resolves to the list of entities `["jodhpur", "hotels"]`.
  - "umaid bhawan palace", which resolves to the list of entities 
    `["umaid bhawan", "palace"]`.
  - "rajasthan travel", which resolves to the list of entities `["rajasthan", "travel"]`
  Then, the resulting temporal simplex constructed is `["jodhpur", "hotels", "umaid bhawan", "palace", "rajasthan", "trave"]`.

4. The resulting simplex is inserted into the simplex tree as a branch in the trie
  structure. As SQLite (the database used in this current implementation) does not support hash indexes, B-tree indexes are used in place. Despite so, the operations defined in the paper are still relatively efficient, with the following new time complexities:
- **Search**: O(j log n) to check if a simplex exists
- **Insert**: O(j log n) to add a new simplex
- **Coface lookup**: O(k T log n) to find all simplices containing given vertices

Due to API rate limits and time constraints, we've processed **1,410 entries** (2.5%) of the full dataset. The pipeline supports checkpointing—it can be resumed at any time to continue processing with the command below.

```bash
cd src
python pipeline.py search_history.json --delay 0.1
```

This partial dataset is sufficient to demonstrate the retrieval pipeline, though gap detection becomes more meaningful with denser coverage.

## The Retrieval Pipeline

The retrieval pipeline (`src/retrieval.py`) processes a query through four stages:

1. The query is embedded using the same model (`openai/text-embedding-3-small`) and compared against all stored vertex embeddings via cosine similarity. Vertices above a threshold form the **query set Q**.

  | Query | Matched Vertices (Q) |
  |-------|---------------------|
  | "What hotel was I looking at in Jodhpur?" | `["Jodhpur", "hotel", "Umaid Bhawan Palace"]` |

2. For each vertex in Q, we find all simplices containing it via coface lookup. If `"Jodhpur"` appears in a 3-simplex `{Jodhpur, Umaid Bhawan, heritage hotel}`, we retrieve that entire co-occurrence pattern. The union of all cofaces forms the **query-induced subcomplex K_Q**. This surfaces related entities the user may have forgotten—context the graph remembers.

3. For each coface, we enumerate its **theoretical faces** (all 2^n - 1 subsets) and check which are missing from the database. Missing faces represent **knowledge gaps**. For example, if we have:
  - 2-simplex `{Jodhpur, Umaid Bhawan, hotel}` (from one session)
  - 1-simplex `{Jodhpur, Mehrangarh Fort}` (from another session)
  - But no simplex containing `{Umaid Bhawan, Mehrangarh Fort}`

  Then `{Umaid Bhawan, Mehrangarh Fort}` is a gap—the user searched for both Jodhpur attractions but never together. The agent can infer a connection but should flag lower confidence.

4. The pipeline assembles context for the LLM: matched vertices with similarity scores, coface contents with their temporal/location context, explicit edges, and knowledge gaps. The `format_context()` method renders this as:

  ```
  === Matched Entities ===
    - Jodhpur (similarity: 0.89)
    - Umaid Bhawan Palace (similarity: 0.72)

  === Co-occurrence Patterns (Simplices) ===
    - [from 2023-05-01T10:30:00 to 2023-05-01T10:45:00] {Jodhpur, Umaid Bhawan Palace, heritage hotel}
    - [at home] {Jodhpur, travel planning, flight booking}

  === Known Relationships ===
    - (Umaid Bhawan Palace) --[located_in]--> (Jodhpur)

  === Knowledge Gaps (Unconfirmed Relationships) ===
    - {Umaid Bhawan Palace, Mehrangarh Fort} - never directly observed together
  ```

  The temporal and location context helps the agent construct a narrative: the user researched Jodhpur hotels during a specific session, and also searched for Jodhpur-related topics from home on other occasions. In practice, one might give explicit instructions to an LLM agent to ingest this information and produce a chain of thought reasoning about the user's narrative before handing over the results to another LLM agent that is responsible for the actual query output. 

## Usage

**Extraction:**
```bash
cd src
python pipeline.py search_history.json --delay 0.1 --window 30
```

**Querying:**
```bash
cd src
python retrieval.py "What hotel was I looking at in Jodhpur?" --top-k 10 --threshold 0.3
```